In [ ]:
Rename SV with blocks

In [ ]:
#!/bin/bash

renamed_folders_count=0

extract_date() {
    local file="$1"
    local date_info=$(strings "$file" | grep "^DATE:" | awk -F "DATE:" '{print $2}')
    local month=$(echo "$date_info" | awk '{print $2}')
    local day=$(echo "$date_info" | awk '{print $3}')
    local year=$(echo "$date_info" | awk '{print $NF}')

    local month_num=$(date -d "$month 1 2000" +%m 2> /dev/null)
    if [ -z "$month_num" ]; then
        echo "Error: Unable to extract date from $file."
        return 1
    fi
    month_num=$((10#$month_num))
    day=$((10#$day))

    printf "%4d%02d%02d" "$year" "$month_num" "$day"
}


rename_files() {
    local session_dir="$1"
    local id="$2"
    local date="$3"
    local block_number=""

    local trial_dat_file=$(find "$session_dir" -maxdepth 1 -name "actual_Trial*FinalR*.dat")
    if [[ -n $trial_dat_file ]]; then
        block_number=$(echo "$trial_dat_file" | grep -oP 'FinalR\K\d+')
    fi

    local edf_file=$(find "$session_dir" -maxdepth 1 -name "*.edf")
    if [[ -n $edf_file && -n $block_number ]]; then
        mv "$edf_file" "${session_dir}/sub-${id}_date-${date}_proj-sceneviewing_task-study_run-${block_number}_eyetrack.edf"
    fi

    if [[ -n $trial_dat_file ]]; then
        mv "$trial_dat_file" "${session_dir}/sub-${id}_${date}_scene_study_run${block_number}.dat"
    fi

    local first_trial_dat_file=$(find "$session_dir" -maxdepth 1 -name "actual_firsttrialFixation*.dat")
    if [[ -n $first_trial_dat_file && -n $block_number ]]; then
        mv "$first_trial_dat_file" "${session_dir}/sub-${id}_${date}_proj-scene_study_run${block_number}_first_fixation.dat"
    fi
}

for id_folder in */; do
    if [[ -d "$id_folder" ]]; then
        id=${id_folder%/}
        for session_folder in "${id_folder}"*/; do
            session_name=$(basename "$session_folder")
            if ! [[ $session_name =~ ^[a-zA-Z0-9]+$ ]]; then
                echo "Folder '$session_folder' has an inconsistent naming format."
                echo -n "Enter the correct folder name (should only contain alphanumeric characters) for '$session_folder': "
                read id
                while ! [[ $id =~ ^[a-zA-Z0-9]+$ ]]; do
                    echo "Invalid ID format. Please enter a valid ID for '$session_folder': "
                    read id
                done
            fi

            if ! [[ $session_name =~ ^[0-9]{8}$ ]]; then
                date=""
                for edf_file in "${session_folder}"*.edf; do
                    if [[ -f "$edf_file" ]]; then
                        extracted_date=$(extract_date "$edf_file")
                        if [[ $? -eq 0 ]]; then
                            date=$extracted_date
                            break
                        fi
                    fi
                done

                if [[ -z $date ]]; then
                    echo "Error: Cannot find date for files in directory '$session_folder'"
                    echo -n "Enter the date for files in '$session_folder' in DDMMYYYY format (e.g. 11122023): "
                    read date
                    while ! [[ $date =~ ^[0-9]{8}$ ]]; do
                        echo "Invalid date format. Please enter a valid date for '$session_folder': "
                        read date
                    done
                fi

                new_session_folder="${id_folder}${date}"
                if [[ ! -d "$new_session_folder" ]]; then
                    mv "$session_folder" "$new_session_folder" && {
                        ((renamed_folders_count++))
                        echo "$session_folder -> $new_session_folder"
                        rename_files "$new_session_folder" "$id" "$date"
                    }
                fi
            fi
        done
    fi
done

echo "Renaming completed."
echo "Total session folders renamed: $renamed_folders_count"